## keras-tuner
- 目的: 用最簡潔的code去找到最佳模型
- [官方](https://keras-team.github.io/keras-tuner/)
- 兩種方式
    - build_model
    - 繼承HyperModel

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch    # 透過RandomSearch來嘗試

In [8]:
# 一個範例: 
# hp: from kerastuner import HyperParameters

def build_model(hp):
    """
        建立一個簡單的分類模型，將超參數設定傳入來將想測試的超參數設定，
        這個函式會被包含在tuner裡面，hp是固定參數
        如: units=hp.Int('units', min_value=32, max_value=512, step=32)
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    """
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dense(units=2, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]),
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )
    return model

In [9]:
# make a instance a kerastuner
# 下面一些必要設定，根據需求去調整

tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_acc',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='keras-tuner-first-try'
)

In [10]:
# 搜尋空間

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
# 這就跟keras model.fit一樣, 開始搜尋

tuner.search(x_train, y_train,
             epochs=5,
             batch_size=128
             validation_data=(x_test, y_test))